# Removing empty containers

Problem: empty containers

```
Group
└── Project
    └── Subject
        └── Session
            └── Acquisition
```

So delete:
* acquisition with no files 
* sessions with no acquisitions
* subjects with no sessions

But files can be attached to any level so make sure there are none.  What else should be checked?  Options to check for certain metadata, tags, presence in a collection?

Needs to be run at various levels but it has to go to the bottom to check for emptyness.  These scripts can be called individually to address a particular level or at the top to get all empty containers.  If running from a gear, run at the level of the destination container.

In [ ]:
import flywheel
fw = flywheel.Client('', root=True)
fw.get_config().site.api_url

In [ ]:
def delete_empty_acqusition(acqusition, dry_run=False):
    print(f'    Checking if acqusition "{acqusition.label}" is empty')
    num_files = len(acqusition.files)
    print(f'    Found {num_files} files')
    acqusition_has_no_files = num_files == 0
    if acqusition_has_no_files:
        if dry_run:
            print(f'    NOT Deleting acqusition "{acqusition.label}"')
        else:
            print(f'    Deleting acqusition "{acqusition.label}"')
            print(fw.delete_acquisition(acquisition.id))
    return acqusition_has_no_files

In [ ]:
def delete_empty_acqusitions(session, dry_run=False):
    acquisitions = session.acquisitions()
    num_acquisitions = len(acquisitions)
    num_deleted = 0
    print(f'Found {num_acquisitions} acquisitions')
    for acquisition in acquisitions:
        if delete_empty_acqusition(acquisition, dry_run):
            num_deleted += 1
    msg = 'Almost ' if dry_run else ""
    print(f'{msg}Deleted {num_deleted}/{num_acquisitions} sessions')
    return num_deleted == num_acquisitions

In [ ]:
def delete_empty_session(session, dry_run=False):
    print(f'  Checking if session "{session.label}" is empty')
    num_files = len(session.files)
    print(f'  Found {num_files} files')
    session_has_no_files = num_files == 0
    all_acqusition_empty = delete_empty_acqusitions(session, dry_run)
    session_empty = all_acqusition_empty and session_has_no_files
    if session_empty:
        if dry_run:
            print(f'  NOT Deleting session "{session.label}"')
        else:
            print(f'  Deleting session "{session.label}"')
            print(fw.delete_session(session.id))
    return session_empty

In [ ]:
def delete_empty_sessions(subject, dry_run=False):
    sessions = subject.sessions()
    num_sessions = len(sessions)
    num_deleted = 0
    print(f'Found {num_sessions} sessions')
    for session in sessions:
        if delete_empty_session(session, dry_run):
            num_deleted += 1
    msg = 'Almost ' if dry_run else ""
    print(f'{msg}Deleted {num_deleted}/{num_sessions} sessions')
    return num_deleted == num_sessions

In [ ]:
def delete_empty_subject(subject, dry_run=False):
    print(f'Checking if subject "{subject.label}" is empty')
    num_files = len(subject.files)
    print(f'Found {num_files} files')
    subject_has_no_files = num_files == 0
    all_sessions_empty = delete_empty_sessions(subject, dry_run)
    subject_empty = all_sessions_empty and subject_has_no_files
    if subject_empty:
        if dry_run:
            print(f'NOT Deleting subject "{subject.label}"')
        else:
            print(f'Deleting subject "{subject.label}"')
            print(fw.delete_subject(subject.id))
    return subject_empty

In [ ]:
def delete_empty_subjects(project, dry_run=False):
    print(f'Deleting empty subjects for project "{project.label}"')
    subjects = project.subjects()
    num_subjects = len(subjects)
    num_deleted = 0
    print(f'Found {len(subjects)} subjects')
    for ii, subject in enumerate(subjects):
        print(f'Subject # {ii:3d} ---------------------------------')
        if delete_empty_subject(subject, dry_run):
            num_deleted += 1
    msg = 'Almost ' if dry_run else ""
    print(f'{msg}Deleted {num_deleted}/{num_subjects} subjects')
    return num_deleted == num_subjects

## Create Project for testing

In [ ]:
group = fw.get('flywheel')
project = group.add_project(label='Deletion Test Project')

In [ ]:
subject = project.add_subject(label='Subject 01')
session = subject.add_session(label='Session 01')
acquisition = session.add_acquisition(label='Localizer')
acquisition.upload_file('FLYWHEEL.dicom.zip')

In [ ]:
delete_empty_subjects(project, dry_run=True)

It didn't delete it because it was not empty (and it was a dry-run)

In [ ]:
delete_empty_subjects(project)

Again it didn't delete becase the acquisiton had a file.  So delete that file.

In [ ]:
acquisition.delete_file('FLYWHEEL.dicom.zip')

In [ ]:
delete_empty_subjects(project, dry_run=True)

In [ ]:
delete_empty_subjects(project)

## Now add some files at different levels to the project to be deleted

In [ ]:
subject = project.add_subject(label='Subject 01')
subject.upload_file('BIDS.png')
session = subject.add_session(label='Session 01')
session.upload_file('words.txt')
acquisition = session.add_acquisition(label='Localizer')
acquisition.upload_file('FLYWHEEL.dicom.zip')
acquisition.upload_file('shme.json')

In [ ]:
delete_empty_subjects(project, dry_run=True)

In [ ]:
delete_empty_subjects(project)

Now delete files and see things get deleted

In [ ]:
acquisition.delete_file('FLYWHEEL.dicom.zip')

In [ ]:
delete_empty_subjects(project)

In [ ]:
acquisition.delete_file('shme.json')

In [ ]:
delete_empty_subjects(project)

In [ ]:
print(session.delete_file('words.txt'))
delete_empty_subjects(project)

In [ ]:
print(subject.delete_file('BIDS.png'))  # file can't be seen in UI because no sessions exist
delete_empty_subjects(project)